In [1]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client, metrics, wait
# wait for jobs to arrive, depending on the queue, this may take some time
import dask.array as da
import dask.bag as db
import numpy as np
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from distributed import progress
import os
from matplotlib import use
use('agg')
os.environ['PROJ_LIB'] = '/home/zsherman/anaconda3/envs/cmac_env/share/proj/'
import pyart
import netCDF4
import tarfile
import tempfile
import shutil
from netCDF4 import num2date
import json
from time import strftime, sleep
import os
%matplotlib inline
import argparse
import datetime
import glob
import importlib
import subprocess
import time

from cmac import (cmac, get_cmac_values, quicklooks,
                  get_sounding_times, get_sounding_file_name)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/home/zsherman/pyart/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/home/zsherman/pyart/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [2]:
def run_cmac_and_plotting(
    radar_file_path, sounding_times, config, clutter_file, out_radar, image_directory, bad_directory, meta_append,
    overwrite, dd_lobes):
    """ For dask we need the radar plotting routines all in one subroutine. """
    cmac_config = get_cmac_values(config)

    try:
        radar = pyart.io.read(radar_file_path)
    except TypeError:
        if bad_directory is None:
            path = os.path.expanduser('~') + '/' + 'type_error_radars/'
        else:
            path = bad_directory
        print(radar_file_path + ' has encountered TypeError!')
        if not os.path.exists(path):
            os.makedirs(path)
            subprocess.call('chmod -R g+rw ' + path, shell=True)
        shutil.move(radar_file_path, path)
        return

    radar_start_date = netCDF4.num2date(radar.time['data'][0],
                                        radar.time['units'])
    year_str = "%04d" % radar_start_date.year
    month_str = "%02d" % radar_start_date.month
    day_str = "%02d" % radar_start_date.day
    hour_str = "%02d" % radar_start_date.hour
    minute_str = "%02d" % radar_start_date.minute
    second_str = "%02d" % radar_start_date.second

    save_name = cmac_config['save_name']
    if out_radar is None:
        the_path = (os.path.expanduser('~') + '/'+ year_str + month_str
                    + second_str + '/')
    else:
        the_path = (out_radar + '/' + year_str +  month_str
                    + day_str)
    file_name = (the_path + '/' + save_name + '.'
                 + year_str + month_str + day_str + '.' + hour_str
                 + minute_str + second_str + '.nc')

    # If overwrite is False, checks to see if the cmac_radar file
    # already exists. If so, CMAC 2.0 is not used on the original radar file.
    if overwrite is False and os.path.exists(file_name) is True:
        print(file_name + ' already exists.')
        return

    if not os.path.exists(the_path):
        os.makedirs(the_path)
        subprocess.call('chmod -R g+rw ' + the_path, shell=True)

    # Load clutter files.
    if clutter_file is not None:
        clutter_file_path = clutter_file
        if verbose:
            print('## Loading clutter file ' + clutter_file_path)
        clutter = pyart.io.read(clutter_file_path)
        if verbose:
            print('## Reading dictionary...')
        clutter_field_dict = clutter.fields['xsapr_clutter']
        if verbose:
            print('## Adding clutter field..')
        try:
            radar.add_field(
                'xsapr_clutter', clutter_field_dict, replace_existing=True)
            del clutter
        except ValueError:
            del clutter
            return

    # Retrieve closest sonde in time to the time of the radar file.
    closest_time = min(
        sounding_times, key=lambda d: abs(d - radar_start_date))
    sonde_name = cmac_config['sonde_name']
    sonde_file = get_sounding_file_name(
        sonde_path, sonde_name, closest_time)
    sonde = netCDF4.Dataset(sonde_file)

        # Running the cmac code to produce a cmac_radar object.
    try:
        cmac_radar = cmac(radar, sonde, config, flip_velocity=True,
                          meta_append=meta_append,
                          verbose=verbose)
    except ValueError:
        del radar
        sonde.close()
        return

    # Free up some memory.
    del radar
    sonde.close()

    # Produce the cmac_radar file from the cmac_radar object.
    pyart.io.write_cfradial(file_name, cmac_radar)
    print('## A CMAC radar object has been created at ' + file_name)

    # Providing the image_directory and checking if it already exists.
    img_directory = (image_directory + '/' + year_str + month_str
                     + day_str + '.' + hour_str + minute_str + second_str)
    if not os.path.exists(img_directory):
        os.makedirs(img_directory)
        subprocess.call('chmod -R g+rw ' + img_directory, shell=True)

    # Producing all the cmac_radar quicklooks.
    quicklooks(cmac_radar, config,
               image_directory=img_directory,
               dd_lobes=dd_lobes)

    # Delete the cmac_radar object and move on to the next radar file.
    del cmac_radar
    return

In [3]:
def get_zdr_values(file):
    try:
        radar = pyart.io.read(file, delay_field_loading=True)
    except OSError:
        return np.nan
    # Check for 40 dBZ echoes above 4 km
    gate_z = radar.gate_z['data']
    ref = radar.fields['reflectivity']['data']
    try:
        snr = radar.fields['signal_to_noise_ratio']['data']
    except KeyError:
        snr = radar.fields['SNR']['data']
    rhohv = radar.fields['cross_correlation_ratio']['data']
    elevation = radar.elevation['data']
    zdr = radar.fields['differential_reflectivity']['data']
    gate_id = radar.fields['gate_id']['data']
    r = radar.range['data']
    which_ranges = np.where(r > 20000.0)[0]
    which_elevations = np.where(elevation > 1)[0]
    above_melting = gate_z > 4000.0
    below_melting = gate_z <= 4000.0
    
    # First test for convective sample - filter out
    num_heavy_gates_aloft = len(np.where(ref[above_melting] > 40.0)[0])
    if num_heavy_gates_aloft > 40:
        return np.nan
    snr_rho_filter = np.logical_and(snr > 20, rhohv > 0.98)
    ref_filtered = ref[snr_rho_filter]
    len_1030 = len(np.where(np.logical_and(ref_filtered >= 10, ref_filtered <= 30))[0])
    len_30 = len(np.where(ref_filtered > 30)[0])
    if(len_30/len_1030 > 0.2):
        return np.nan
    
    zdr = np.ma.masked_where(gate_id != 1, zdr)
    ref_filtered = np.ma.masked_where(gate_id != 1, ref)

    zdr = zdr[which_elevations, :]
    zdr = zdr[:, which_ranges]
    ref_filtered = ref_filtered[which_elevations, :]
    ref_filtered = ref_filtered[:, which_ranges]
    
    del radar
    z_bins = np.linspace(19, 30.5, 7)
    mean_zdr = np.zeros(len(z_bins)-1)
    for i in range(len(z_bins)-1):
        inds = np.where(np.logical_and(ref_filtered >= z_bins[i], ref_filtered <= z_bins[i+1]))
        print(inds)
        if(len(inds[0]) > 200):
            mean_zdr[i] = np.ma.mean(zdr[inds])
        else:
            mean_zdr[i] = np.nan
    return np.nanmean(mean_zdr)

In [4]:
def add_zdr_offset(file, zdr_offset):
    try:
        radar = pyart.io.read(file)
        radar.fields['corrected_differential_reflectivity']['data'] += zdr_offset
        pyart.io.write_cfradial(file, radar)
        del radar
        return
    except (PermissionError, OSError) as e:
        try:
            del radar
            return
        except NameError:
            return

In [5]:
radar_path = '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI5.00/201710'
sonde_path = '/lustre/or-hydra/cades-arm/proj-shared/sgpsondewnpnC1.b1/'
config = 'xsapr_i5_ppi'

In [6]:
cmac_config = get_cmac_values(config)

x_compass = cmac_config['x_compass']
if os.path.isdir(radar_path):
    radar_files = glob.glob(radar_path + '/**/*' + x_compass
                            + '*', recursive=True)
elif os.path.isfile(radar_path):
    with open(radar_path) as f:
        radar_files = f.readlines()
    radar_files = [x.strip() for x in radar_files]
else:
    raise IOError('The specified radar path does not exist!')
sonde_name = cmac_config['sonde_name']
sounding_times = get_sounding_times(sonde_path, sonde_name)

# Get dates of radar files from the file name.
radar_times = []
for file_name in radar_files:
    where_x = file_name.find(x_compass)
    radar_times.append(
        datetime.datetime.strptime(file_name[where_x+3:where_x+15],
                                   '%y%m%d%H%M%S'))

In [7]:
radar_files

['/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI5.00/201710/sgpxsaprI5.00.20171019.061101.mnt.XSW171019052640.RAWS4AR.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI5.00/201710/sgpxsaprI5.00.20171022.151101.mnt.XSW171022142049.RAWS4U6.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI5.00/201710/sgpxsaprI5.00.20171031.111101.mnt.XSW171031095846.RAWS642.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI5.00/201710/sgpxsaprI5.00.20171013.011101.mnt.XSW171012235425.RAWS3DA.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI5.00/201710/sgpxsaprI5.00.20171012.111101.mnt.XSW171012101313.RAWS3AH.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI5.00/201710/sgpxsaprI5.00.20171009.091102.mnt.XSW171009081005.RAWCLHV.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI5.00/201710/sgpxsaprI5.00.20171006.091102.mnt.XSW171006085004.RAWCL4W.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprpp

In [8]:
#cluster = PBSCluster(name='dask-worker', memory='270GB', cores=36, processes=6, interface='ib0', queue='high_mem', project='arm',
#                    walltime='00:30:00')#, job-extra=['-W group_list=cades-arm'])
cluster1 = PBSCluster(processes=12, cores=12, walltime='3:00:00', queue='batch', memory='128GB')
cluster1.scale(12*12)         # Ask for ten workers
client1 = Client(cluster1)  # Connect this local process to remote workers

In [9]:
cluster1

In [11]:
client1

Client Scheduler: tcp://10.23.216.82:43398 Dashboard: http://10.23.216.82:8787/status,Cluster Workers: 144 Cores: 144 Memory: 1.54 TB


In [12]:
def com():
    os.environ['PROJ_LIB'] = '/home/zsherman/anaconda3/envs/cmac_env/share/proj/'
    from mpl_toolkits.basemap import Basemap
    return os.environ['PROJ_LIB']
client1.run(com)

{'tcp://10.23.216.159:33217': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:34443': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:35261': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:36652': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:36776': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:36790': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:38094': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:38402': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:41054': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:42393': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:46418': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:46421': '/home/zsherman/anaconda3/envs/cmac_env/share/

In [13]:
bad_directory = None
out_radar = '/lustre/or-hydra/cades-arm/proj-shared/sgpxsaprcmacsurI5.c1'
image_directory = '/lustre/or-hydra/cades-arm/proj-shared/sgpxsaprcmacsurI5.c1.png'
meta_append = 'config'
overwrite = True
clutter_file = '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI5.00/clutter_files/clutter_201710.nc'
verbose = False
dd_lobes = True

In [14]:
#the_bag = db.from_sequence(radar_files)
the_function = lambda x: run_cmac_and_plotting(
    x, sounding_times=sounding_times, config='xsapr_i5_ppi',
    clutter_file=clutter_file, out_radar=out_radar, image_directory=image_directory,
    bad_directory=bad_directory, meta_append=meta_append,
    overwrite=overwrite, dd_lobes=dd_lobes)
futures = client1.map(the_function, radar_files)

In [15]:
#start computation in the background
progress(futures)

VBox()

In [17]:
futures

[<Future: status: finished, type: NoneType, key: <lambda>-342185b49bd724ae9c65877c73140f25>,
 <Future: status: finished, type: NoneType, key: <lambda>-e5465a49e20b4fc88628027c7f369674>,
 <Future: status: finished, type: NoneType, key: <lambda>-a9eefd165f0181c0fcf3b129c34fd339>,
 <Future: status: finished, type: NoneType, key: <lambda>-dc1ef9d3adb42e2d548398cc04e01238>,
 <Future: status: finished, type: NoneType, key: <lambda>-ff30fd5584b81e0ff4d711be7acb7904>,
 <Future: status: finished, type: NoneType, key: <lambda>-916c94647bc8b26e4d678833d142a39a>,
 <Future: status: finished, type: NoneType, key: <lambda>-f10f38171cd5195463074b1febf7fd03>,
 <Future: status: finished, type: NoneType, key: <lambda>-1808beea070022a4d2c86cd871a68297>,
 <Future: status: finished, type: NoneType, key: <lambda>-5cfe962637556327377de1d1c1614a2f>,
 <Future: status: finished, type: NoneType, key: <lambda>-3d697997a5e33906a2a4125d332e8e73>,
 <Future: status: finished, type: NoneType, key: <lambda>-b304cce60596

In [18]:
cluster1.stop_all_jobs()

In [19]:
radar_path = '/lustre/or-hydra/cades-arm/proj-shared/sgpxsaprcmacsurI5.c1/201710**/*'

In [20]:
radar_files = glob.glob(radar_path, recursive=True)

In [21]:
#cluster = PBSCluster(name='dask-worker', memory='270GB', cores=36, processes=6, interface='ib0', queue='high_mem', project='arm',
#                    walltime='00:30:00')#, job-extra=['-W group_list=cades-arm'])
cluster2 = PBSCluster(processes=8, cores=8, walltime='3:00:00', queue='batch', memory='128GB')
cluster2.scale(8*8)         # Ask for ten workers
client2 = Client(cluster2)  # Connect this local process to remote workers

/home/zsherman/anaconda3/envs/cmac_env/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [24]:
cluster2

In [25]:
client2

Client Scheduler: tcp://10.23.216.82:38117 Dashboard: http://10.23.216.82:37026/status,Cluster Workers: 56 Cores: 56 Memory: 896.00 GB


In [26]:
def com():
    os.environ['PROJ_LIB'] = '/home/zsherman/anaconda3/envs/cmac_env/share/proj/'
    from mpl_toolkits.basemap import Basemap
    return os.environ['PROJ_LIB']
client2.run(com)

{'tcp://10.23.216.116:33332': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.116:33599': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.116:35055': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.116:39126': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.116:39485': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.116:39863': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.116:40345': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.116:41674': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:35936': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:36221': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:39888': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.159:42881': '/home/zsherman/anaconda3/envs/cmac_env/share/

In [27]:
file_list = sorted(radar_files)
the_bag = db.from_sequence(file_list)
the_function = lambda x: get_zdr_values(x)
biases = the_bag.map(get_zdr_values).compute()

In [28]:
arr_biases = np.array(biases)

In [29]:
zdr_offset = np.nanmean(arr_biases)

In [30]:
zdr_offset

-3.1004412109014368

In [31]:
the_function = lambda x: add_zdr_offset(x, zdr_offset=zdr_offset)
add_zdr = client2.map(the_function, file_list)

In [32]:
progress(add_zdr)

VBox()

In [ ]:
cluster2.stop_all_jobs()

In [ ]:
add_zdr